In [11]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [12]:
%sql sqlite:///results.db

# Renaming commands (Nicknaming)

`A = B`

Renaming occurs when a new name is given to a command, but no semantic changes are introduced (i.e. the alias value is simply the command name, nothing else).

In [13]:
%%sql 
with total as (select count(*) as total from alias)
select count(*) as '#', round(count(*)*100.0/total,2) as '%' 
from alias join command using (alias_id)
join total
where alias.value = command.name
and alias.name != command.name
and alias.name not in known_commands

 * sqlite:///results.db
Done.


#,%
244872,11.11


In [15]:
%%sql
select alias.name, alias.value, count(*) as '#'
from alias join command using (alias_id)
where alias.value = command.name
and alias.name != command.name
and alias.name not in known_commands
group by alias.value, alias.name
order by count(*) desc
limit 20;

 * sqlite:///results.db
Done.


name,value,#
g,git,8618
c,clear,6815
h,history,4376
v,vim,3972
cls,clear,3047
l,ls,2702
dc,docker-compose,2331
sl,ls,2074
cl,clear,1891
k,kubectl,1778


In [28]:
%%sql
select count(*) as '#'
from alias join command using (alias_id)
where alias.value = command.name
and alias.name != command.name
and alias.name not in known_commands
and length(alias.name) <= 2;

 * sqlite:///results.db
Done.


#
124518


In [31]:
%%sql
select count(*) as '#'
from alias join command using (alias_id)
where alias.value = command.name
and alias.name != command.name
and alias.name not in known_commands
and length(alias.name) <= length(alias.value)

 * sqlite:///results.db
Done.


#
227802


---
## By Command

In [32]:
%%sql
select 
    cmd_name,
    count(distinct alias_id) as num,
    round(count(distinct alias_id)*100.0/total,2) as per
from (
    select command.name as cmd_name, alias_id
    from alias join command using (alias_id)
    where alias.value = command.name
    and alias.name != command.name
    and alias.name not in known_commands
)
join command_alias_totals using (cmd_name)
group by cmd_name
having num > 1000 and per > 25
order by per desc, num desc

 * sqlite:///results.db
Done.


cmd_name,num,per
exit,6562,85.86
clear,13448,68.29
popd,1554,64.75
ranger,1429,58.98
history,4934,44.55
jobs,1231,40.99
pushd,1356,38.05
xdg-open,1298,37.16
sudo,2971,34.35
docker-compose,3543,32.99
